<h1>Exploratory Data Analysis: Email<br>
    <span style='font-weight: medium; font-size:1.5rem; line-height:1.05'>Part 1: Cleaning and reorganizing&nbsp;data</span>
<h1>

In [1]:
# Converting dataset to a pandas dataframe
# Reading data in chunks because the dataset is large

import pandas as pd

file = '../input/enron-email-dataset/emails.csv'
ROWS_PER_CHUNK = 1000
df = pd.DataFrame({})

reader = pd.read_csv(
file,
chunksize=ROWS_PER_CHUNK,
iterator=True
)

for data_chunk in reader:
    df = df.append(data_chunk)

In [2]:
# Getting details about the data

df.shape

(517401, 2)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB


In [4]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [5]:
# Viewing one of the messages in the message column

print(df.message[10])

Message-ID: <33076797.1075855687515.JavaMail.evans@thyme>
Date: Mon, 16 Oct 2000 06:42:00 -0700 (PDT)
From: phillip.allen@enron.com
To: buck.buckner@honeywell.com
Subject: Re: FW: fixed forward or other Collar floor gas price terms
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: "Buckner, Buck" <buck.buckner@honeywell.com> @ ENRON
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

Mr. Buckner,

 For delivered gas behind San Diego, Enron Energy Services is the appropriate 
Enron entity.  I have forwarded your request to Zarin Imam at EES.  Her phone 
number is 713-853-7107.  

Phillip Allen


The message column contains all the information that we need for the exploratory data analysis (EDA). But we need to organize this information into distinct columns so that we can visualize and study it easily.

Let us load and then apply a few functions to extract and organize information from the message column. These functions are slightly modified versions of functions that have been used in the notebook [Enron Network Analysis](https://www.kaggle.com/jamestollefson/enron-network-analysis).

In [6]:
# %load ../usr/lib/utilities_email/utilities_email.py
import re

def standard_format(df, column, text, num):
    """This function drops dataframe rows that do not contain specific text."""
    rows = []
    for row, message in enumerate(column):
        message_words = message.split('\n')
        if text not in message_words[num]:
            rows.append(row)
    df = df.drop(df.index[rows])
    return df

def get_text(column, row_num_slicer):
    """This function returns a pandas series that contains a snippet retrieved from a
    larger chunk of text. The snippet is obtained by deleting preceding rows from the chunk."""
    result = pd.Series(index=column.index, dtype='object')
    for row, message in enumerate(column):
        message_words = message.split('\n')
        del message_words[:row_num_slicer]
        result.iloc[row] = message_words
    return result

def get_row(column, row_num):
    """This function returns a pandas series that consists of a row sliced from a larger chunk of text."""
    result = pd.Series(index=column.index, dtype='object')
    for row, message in enumerate(column):
        message_words = message.split('\n')
        message_words = message_words[row_num]
        result.iloc[row] = message_words
    return result

def get_address(df, column, num_cols=1):
    """returns a specified email address from each row in a Series"""
    address = re.compile('[\w\.-]+@[\w\.-]+\.\w+')
    addresses = []
    result1 = pd.Series(index=df.index, dtype='object')
    result2 = pd.Series(index=df.index, dtype='object')
    result3 = pd.Series(index=df.index, dtype='object')
    for i in range(len(df)):
        for message in column:
            correspondents = re.findall(address, message)
            addresses.append(correspondents)
            result1[i] = addresses[i][0]
        if num_cols >= 2:
            if len(addresses[i]) >= 3:
                result2[i] = addresses[i][1]
                if num_cols == 3:
                    if len(addresses[i]) >= 4:
                        result3[i] = addresses[i][2]
    return result1, result2, result3


In each field of the message column of the dataframe, lines that begin with specific words are relevant to us. Let us save these words in a&nbsp;list.

In [7]:
headers = ['Message-ID: ', 'Date: ', 'From: ', 'To: ', 'Subject: ']

Next, let us use the `standard_format()` function to drop rows of the message column that do not contain any one of the words in the `headers`&nbsp;list.

In [8]:
import numpy as np

x = len(df.index)

for i, v in enumerate(headers):
    df = standard_format(df, df.message, v, i)
    
df = df.reset_index(drop=True)
num_deleted = x - len(df.index)
percent_deleted = np.round(((x - len(df.index)) / x)\
                                           * 100, decimals=2)

print(f"Deleted {num_deleted} irrelevant messages... \
That's {percent_deleted}% of the total number of messages in the dataset.")

Deleted 111433 irrelevant messages... That's 21.54% of the total number of messages in the dataset.


In [9]:
# Reducing the number of rows in the dataframe
# df = df[:200000]

Now, we will extract relevant data from the message column and save the data in new columns within the dataframe.

In [10]:
# Creating new columns
df['date'] = get_row(df.message, 1)
df['sender'] = get_row(df.message, 2)
df['recipients'] = get_row(df.message, 3)
df['subject'] = get_row(df.message, 4)
df['text'] = get_text(df.message, 15)

Next, let us look at the first few rows of the updated dataframe as well as its structure.

In [11]:
df.head(3)

,file,message,date,sender,recipients,subject,text
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,"Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)",From: phillip.allen@enron.com,To: tim.belden@enron.com,Subject:,"[, Here is our forecast, , ]"
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,"Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)",From: phillip.allen@enron.com,To: john.lavorato@enron.com,Subject: Re:,"[, Traveling to have a business meeting takes ..."
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,"Date: Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",From: phillip.allen@enron.com,To: leah.arsdall@enron.com,Subject: Re: test,"[, test successful. way to go!!!]"


We will make a few further edits to the dataframe.

In [12]:
# Deleting the first two unneeded columns
del df['file']
del df['message']

In [13]:
# Removing extraneous prefixes from the date and subject columns
df.date = df.date.str.replace('Date: ', '')
df.subject = df.subject.str.replace('Subject: ', '')

In [14]:
# Changing the data type of the date column
df['date'] = df['date'].apply(lambda x: pd.to_datetime(x, errors='coerce', utc=True))

In [15]:
# Changing the time zone of the date column
df['date'] = df.date.dt.tz_convert('Asia/Kolkata')

In [16]:
# Adding columns that specify the day of the week in nominal and numerical formats
df.insert(1, 'day_of_week', df.date.dt.day_name())
df.insert(2, 'day_of_week_num', df.date.dt.weekday)

In [17]:
# Adding a time-of-day column
df.insert(3, 'time_of_day', df.date.apply(lambda x: x.hour + x.minute/60\
                                          + x.second/3600))

In [18]:
# Adding the hour
df.insert(4, 'hour', df.date.dt.hour)

In [19]:
# Adding the year in float format, accounting for number of days elapsed
df.insert(5, 'year', df.date.apply(lambda x: x.year +\
                                   x.dayofyear/365.25))

In [20]:
# Adding the year in integer format 
df.insert(6, 'year_int', df.date.dt.year)

In [21]:
# Assigning the values of the date column to the dataframe index and
# deleting the date column
df.index = df.date
del df['date']

In [22]:
# Removing extraneous prefixes from the recipients and sender columns
df.sender = df.sender.str.replace('From: ', '')
df.recipients = df.recipients.str.replace('To: ', '')

In [23]:
# Calculating IQR so as to remove outliers
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

In [24]:
# Making a backup copy of the dataframe
df_original = df.copy()

In [25]:
# Removing outliers
df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

In [26]:
# Converting all text elements to lowercase
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].astype(str).str.lower().str.strip()

In [27]:
# Sorting the dataframe by year
df = df.sort_values(by=['year'])

In [28]:
# Removing emails with no subject lines
df = df[df['subject'] != '']

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 371841 entries, 1999-08-03 15:34:00+05:30 to 2002-07-12 16:03:00+05:30
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   day_of_week      371841 non-null  object 
 1   day_of_week_num  371841 non-null  int64  
 2   time_of_day      371841 non-null  float64
 3   hour             371841 non-null  int64  
 4   year             371841 non-null  float64
 5   year_int         371841 non-null  int64  
 6   sender           371841 non-null  object 
 7   recipients       371841 non-null  object 
 8   subject          371841 non-null  object 
 9   text             371841 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 31.2+ MB


In [30]:
df.tail()

,day_of_week,day_of_week_num,time_of_day,hour,year,year_int,sender,recipients,subject,text
date,,,,,,,,,,
2002-07-12 16:43:00+05:30,friday,4,16.716667,16,2002.528405,2002,denise.williams@enron.com,ge_benefits@enron.com,urgent!!! cutover weekend,"['', 'it is vital that you leave your computer..."
2002-07-12 14:01:00+05:30,friday,4,14.016667,14,2002.528405,2002,mark.fisher@enron.com,tom.nemila@enron.com,wr613 pitch system performance,"['x-origin: fischer-m', 'x-filename: mark fisc..."
2002-07-12 13:51:00+05:30,friday,4,13.850000,13,2002.528405,2002,mark.fisher@enron.com,tom.nemila@enron.com,wr627 fault paretos (may 2002),"['x-origin: fischer-m', 'x-filename: mark fisc..."
2002-07-12 08:17:00+05:30,friday,4,8.283333,8,2002.528405,2002,nancy.oehring@enron.com,ge_benefits@enron.com,abra hris system,"['', 'this message is for all abra users:', ''..."
2002-07-12 16:03:00+05:30,friday,4,16.050000,16,2002.528405,2002,kurt.anderson@enron.com,"jeff.duff@enron.com, mark.walker@enron.com, ma...",fw: re: revised availability numbers,"['', 'lets get together to discuss..', '------..."


In [31]:
df.to_csv('emails_cleaned.csv')